# Personal Information
Name: **Lucas Belderink**

StudentID: **12151750**

Email: [**lucas.belderink@student.uva.nl**](lucas.belderink@student.uva.nl)

Submitted on: **22.3.2024**

github link: [**https://github.com/Looqes/pca_thesis_project**](https://github.com/Looqes/pca_thesis_project)

# Data Context

The data used in this project consists of sets of images for 170 different patients, acquired from different scanners at different times. The data belongs to the NKI and is aquired from past patients. The images come in the form of .nii files, a common format for multidimensional medical imaging data and are 3-dimensional, thus layered. Also added to the data are .nrrd delineation files which carry the tumor delineations. These are made by pathology within the NKI following radical prostatectomies (in which the prostate is removed). Finally, a folder is included containing tumor delineations made by expert radiologists based on their interpretation of MRI's. These will be used at a later time however as the main focus lies on just using the mri's to train a model to make delineations. For now these will not be loaded and/or used.

# Data Description

For each patient the relevant data (at least at this moment) consists of the following:
+ axial-plane T2 scan
+ A Diffusion Weighted Imaging (DWI) Apparent Diffusion Coefficient (ADC) map
+ A DWI Perfusion map

The axial plane T2 scan is relevant because the other images are also in the axial plane, and the objective is to match and overlap them to then feed them to the model. A python package called nibabel is used to open them & gather their image data as numerical arrays, and other metadata. The first goal is to see how the resolutions of the images vary across patients, as it is apparent beforehand by manual inspection that they vary slightly between patients. To use the data with the model the image have to be resized to a common resultion per patient. The resolution between patients does not have to be the same necessarily as the model planned to be used (nnUnet) does some preprocessing automatically and will take care of this. \

For the delineations, for every patient, a number of .nrrd files is included containing the delineations mapped to the T2 axial scans. When these are opened in conjunction using visualization software in the form of Slicer they overlap and create information about the position and presence of a tumor. They are mapped specifically to the T2 scans for the patients and thus share their resolution. Not all slices of a patient that does actually have a tumor contain delineations however, as the tumor usually does not span the entire prostate. Thus the set of slices that have mapped tumor delineations, in case of the presence of a tumor, is always a subset of the total amount of slices in a patient.

In [ ]:
%pip install -r requirements.txt

# EDA

In [ ]:
import sys
"../scripts" not in sys.path and sys.path.insert(0, '../scripts')

In [ ]:
import importlib
# import os
from os import listdir

from collections import defaultdict
import numpy as np
import pandas as pd
import pylab as plt
# import matplotlib as plt
%matplotlib inline
import SimpleITK as sitk
import nibabel as nb

# Custom objects included in scripts
import patient
import read_scans_utils

In [ ]:
# The amount of patients available
img_folders = [f for f in listdir("../data/Scans")]
print(len(img_folders))

### Taking a look at imaging data for a patient

In [ ]:
importlib.reload(sys.modules["patient"])
importlib.reload(sys.modules["read_scans_utils"])
import patient
import read_scans_utils

In [ ]:
# Create a dict mapping patient id to data including: seriesnumber of AxialT2 scan, name of sequence,
# expected number of slices by manual inspection using Slicer
# Required for reading the axial T2w scans of the patients
seriesnumber_dict = read_scans_utils.load_series_numbers_dict("../data/T2w_seriesnumber_info_Lucas.xlsx")

In [ ]:
# Read sample patient
patient7 = read_scans_utils.read_patient("MARPROC007_nii", seriesnumbers_dict=seriesnumber_dict, print_errors=True)

In [ ]:
# Extract & plot an axial T2 image
axialt2_p7 = patient7.axialt2

# Print the shape of the image
# x, y, z, where x, y is the resolution of a single slice of the 3d image, and z the amount of slices
print(np.asanyarray(axialt2_p7.dataobj).shape)
# print the first slice in the image
plt.imshow(np.asanyarray(axialt2_p7.dataobj)[:, :, 0])

In [ ]:
adcmap_p7 = patient7.adcmap

# ADC maps have a different resolution, and a different amount of slices per patient
# For use in the training of a model, they will need to be resized to a common size
print(np.asanyarray(adcmap_p7.dataobj).shape)

# Plot adcmap image
plt.imshow(np.asanyarray(adcmap_p7.dataobj[:, :, 0]))

In [ ]:
perffrac_p7 = patient7.perfusionmap

# Perfusion maps have the same resolution as ADC maps
print(np.asanyarray(perffrac_p7.dataobj).shape)

# Plot a perfusionmap image
plt.imshow(np.asanyarray(perffrac_p7.dataobj[:, :, 0]))

In [ ]:
# Each of the images are .nii or NifTi files, meaning they carry alot more metadata next to the actual imaging data
print(patient7.axialt2)
# print(patient7.adcmap)
# print(patient7.perfusionmap)

### Checking consistency across the whole set

First, the entire set will have to be read

In [ ]:
patients, erroneous_patient_ids = read_scans_utils.read_patients(scans_data_path="../data/Scans",
                                          seriesnumber_info_path="../data/T2w_seriesnumber_info_Lucas.xlsx")

Now to investigate how the resolutions of imaging differ between different patients the distribution of resolutions will be examined

In [ ]:
from IPython.display import display_html

t2_resolutions = defaultdict(int)
adc_resolutions = defaultdict(int)
perf_resolutions = defaultdict(int)

for patient_id in patients:
    patient = patients[patient_id]

    t2_resolutions[np.asarray(patient.axialt2.dataobj).shape] += 1

    adc_resolutions[np.asarray(patient.adcmap.dataobj).shape] += 1

    perf_resolutions[np.asarray(patient.perfusionmap.dataobj).shape] += 1

    

df1 = pd.DataFrame(t2_resolutions.items(), columns = ["Resolution", "Occurrences"]).sort_values("Occurrences", ascending = False)
df2 = pd.DataFrame(adc_resolutions.items(), columns = ["Resolution", "Occurrences"]).sort_values("Occurrences", ascending = False)
df3 = pd.DataFrame(perf_resolutions.items(), columns = ["Resolution", "Occurrences"]).sort_values("Occurrences", ascending = False)

# read_scans_utils.display_side_by_side(df1, df2, df3)

df1_styler = df1.style.set_table_attributes("style='display:inline'").set_caption('T2 shapes')
df2_styler = df2.style.set_table_attributes("style='display:inline'").set_caption('ADC shapes')
df3_styler = df3.style.set_table_attributes("style='display:inline'").set_caption('Perfusion shapes')
    
display_html(df1_styler._repr_html_()+df2_styler._repr_html_()+df3_styler._repr_html_(), raw=True)

# display(df)

Most of the scans have a consistent resolution, with slight variation in a part of the data.
There are however some variations in either the resolution, the amount of slices, or both.\
Also important to note, is that the perfusion scans and adc scans overlap in their resolutions and slice amounts exactly, meaning the only thing that remains for scaling them for use in the network is to match them to their accompanying T2w scans together. \


In [ ]:
# Check adc shape vs perffrac shape just to be sure
for patient_id in patients:
    patient = patients[patient_id]
    adc = patient.get_adc_image_array()
    perffrac = patient.get_perfusion_image_array()

    if adc.shape != perffrac.shape:
        print("DWI's dont match for ", patient_id)
        print("adc shape      : ", adc.shape)
        print("perfusion shape: ", perffrac.shape)


## Resizing the DWI images

In [ ]:
print("Shapes of array for patient7: ")
patient7_t2 = patient7.get_axialt2_image_array()
patient7_adc = patient7.get_adc_image_array()
patient7_perffrac = patient7.get_perfusion_image_array()
print(patient7_t2.shape)
print(patient7_adc.shape)
print(patient7_perffrac.shape)

the shapes of the DWI's (176, 176, 20) need to be resized to match the T2 image (512, 512, 30) \
What the middle adc slice looks like before resizing:

In [ ]:
patient7_adc = patient7.get_adc_image_array()
plt.imshow(patient7_adc[:, :, int(patient7_adc.shape[2]/2)])

### Resizing using scipy's zoom

In [ ]:
import scipy
from scipy.ndimage import zoom

In [ ]:
x_y_factor = patient7_t2.shape[0]/patient7_adc.shape[0]
z_factor = patient7_t2.shape[2]/patient7_adc.shape[2]

new_image = zoom(patient7_adc, (x_y_factor, x_y_factor, z_factor))
print(new_image.shape)


What the middle slice of the adc image looks like after resizing:

In [ ]:
plt.imshow(new_image[:, :, int(new_image.shape[2]/2)])

### Resizing using nilearn's resample_to_image
This takes into account the affine matrix of the target

In [ ]:
# print(patient7.adcmap)
import nilearn.image as ni_im
import os

resampled_image = ni_im.resample_to_img(patient7.adcmap, patient7.axialt2)
# nb.save(resampled_image, os.path.join(".", 'resampled_image.nii'))

# resampled_image = ni_im.resample_to_img(patient7.adcmap, patient7.axialt2.affine)
# print(patient7.axialt2.affine)
# print(patient7.axialt2.shape)

# resampled_image = ni_im.resample_img(patient7.adcmap, patient7.axialt2.affine, patient7.axialt2.shape)


In [ ]:
print(resampled_image)

In [ ]:
print(resampled_image.shape)
# plt.imshow(resampled_image.dataobj[:, :, int(resampled_image.shape[2]/2)])
plt.imshow(resampled_image.dataobj[:, :, 10])


### Resizing using simpleitk

In [ ]:
# print(patient7.axialt2)

patient7_sitk_t2 = sitk.ReadImage("../data/Scans/MARPROC007_nii/501_tt2_tse.nii")
axial7_array = sitk.GetArrayFromImage(patient7_sitk_t2)
plt.imshow(axial7_array[int(axial7_array.shape[0]/2), :, :])


In [ ]:
patient7_sitk_ADC = sitk.ReadImage("../data/Scans/MARPROC007_nii/899_tdwi_ssepi_801__20140825_adc.nii")
ADC_array = sitk.GetArrayFromImage(patient7_sitk_ADC)
plt.imshow(ADC_array[int(ADC_array.shape[0]/2), :, :])



In [ ]:
Dimension = patient7_sitk_ADC.GetDimension()

Transform = sitk.AffineTransform(Dimension)
Transform.SetMatrix(patient7_sitk_ADC.GetDirection())
Transform.SetTranslation(np.array(patient7_sitk_ADC.GetOrigin()) - patient7_sitk_t2.GetOrigin())

ResImage = sitk.Resample(patient7_sitk_ADC, patient7_sitk_t2, Transform, sitk.sitkLinear, patient7_sitk_ADC.GetPixelIDValue())

In [ ]:
print(ResImage.GetDimension())
print(ResImage.GetSize())
ResImage.GetSpacing()
resampled_array = sitk.GetArrayFromImage(ResImage)
plt.imshow(resampled_array[int(resampled_array.shape[0]/2), :, :])

In [ ]:
resample = sitk.ResampleImageFilter()
resample.SetInterpolator = sitk.sitkLinear
resample.SetOutputDirection = patient7_sitk_ADC.GetDirection()
resample.SetOutputOrigin = patient7_sitk_ADC.GetOrigin()
new_spacing = [1, 1, 1]
print(new_spacing)
resample.SetOutputSpacing(new_spacing)
# new_spacing = np.array(new_spacing)

orig_size = np.array(patient7_sitk_ADC.GetSize(), dtype=np.int64)
orig_spacing = np.array([x for x in patient7_sitk_ADC.GetSpacing()])
print(orig_spacing)
new_size = orig_size*(orig_spacing/new_spacing)
new_size = np.ceil(new_size).astype(np.int64) #  Image dimensions are in integers
new_size = [int(s) for s in new_size]
print(new_size)
resample.SetSize(new_size)

newimage = resample.Execute(patient7_sitk_ADC)

In [ ]:
resampled_array = sitk.GetArrayFromImage(newimage)
plt.imshow(resampled_array[int(resampled_array.shape[0]/2), :, :])

### Opening the delineation files

In [ ]:
importlib.reload(sys.modules["patient"])
importlib.reload(sys.modules["read_scans_utils"])
import patient
import read_scans_utils

In [ ]:
# Reads delineations in the format: {patient_id: [(name_of_file, [array, metadata])
#                                                 (name_of_file, ......           )
#                                                ]
#                                   }
delineations = read_scans_utils.read_delineations()

print(len(delineations))
print(len(delineations["MARPROC007"]))
print([x[0] for x in delineations["MARPROC007"]])
print(delineations["MARPROC007"][0][1][1])

In [ ]:
print(type(delineations["MARPROC007"][1][1][1]))
print(delineations["MARPROC007"][1][1][1])

In [ ]:
foldername_delins = "../data/Regions ground truth/Regions delineations/"

delin_folders = [f for f in listdir(foldername_delins)]
print("Amount of patient folders included in the dataset: ", len(delin_folders))

delineations_counts = defaultdict(int)
patients_to_skip = {x for x in erroneous_patient_ids}
print(patients_to_skip)

# Count the amount of delineation files per patient to see what is the distribution of file counts
for patient_id, patient_delineations in delineations.items():
    if patient_id in patients_to_skip:
        continue
    
    delineations_counts[(len(patient_delineations))] += 1


del_counts_df = pd.DataFrame(delineations_counts.items(), columns = ["Amount of delineations", "count"]).sort_values("count", ascending=False)
display(del_counts_df)
print(del_counts_df["count"].sum())

Most patients have 2 delineation files. There are a few outliers with 5 and even 6 files.

In [ ]:
# patient = None
# delineation = None

# print("Delineations for single patients: ")
# i = 0
# for patient_id, delineations_patient in delineations.items():
#     print(patient_id, [delineations_patient[i][0] for i in range(len(delineations_patient))])
#     i += 1
#     if i > 4:
#         break


# nrrd_delineations = delineations["MARPROC007"]
# nrrd_delineations2 = delineations["MARPROC343"]
# print("\nDelineation data for a single patient:")
# print(nrrd_delineations[0][0])
# print(nrrd_delineations[0][1][0].shape)
# print(nrrd_delineations[0][1][1])
# print()
# print(nrrd_delineations2[0][0])
# print(nrrd_delineations2[0][1][0].shape)
# print(nrrd_delineations2[0][1][1])


# patient_6files = delineations["MARPROC204"]
# print("\nThe files in the folder of the patient with the most files (6):")
# print([patient_6files[i][0] for i in range(len(patient_6files))])

# # Printing 2 files for patient 204 of different zones for GG3 regions
# # They should match in resolution
# print("\nResolution of GG3 in TZ and GG3 in PZ of patient 204:")
# patient_204_delins = delineations["MARPROC204"]
# for delin in patient_204_delins:
#     if "GG3" in delin[0]:
#         print(delin[0])
#         print(delin[1][0].shape, "\n")


Patients have a combination of files representing regions of differing GG tissue. Also, as visible from the patient with the most delineation files available across all patients multiple files are available for even a single GG. This means that the tumor that is delineated for this patient, and its GG regions, cover the two different zones of the prostate: the peripheral zone (PZ) and the transition zone (TZ). For this research the seperation between zones is not relevant, so these will be combined. As visible above in this case they share resolution, since they are both still registered to the entire T2 scan. A total GG3 region delineation will be formed by performing an OR operation of the files for both zones.
As visible the delineation, when opened using pynrrd consists of a 3d image in the form of a 3d numerical array, matching the resolution of the T2 axial image loaded earlier. Also some metadata is included. 

To check the distribution of resolutions of the files:

In [ ]:
delin_resolutions = defaultdict(int)

for patient_id, delineations_patient in delineations.items():
    shapes = set()

    for delineation in delineations_patient:
        shapes.add(delineation[1][0].shape)
    if len(shapes) == 1:
        delin_resolutions[shapes.pop()] += 1


delin_resolutions_df = pd.DataFrame(delin_resolutions.items(), columns=["resolution", "count"])
display(delin_resolutions_df)

The delineations seem to contain some differing resolutions as there are more unique resolutions (20) than the amount of resolutions in the T2 axial scans (14). Some additional checks will have to be done when adding the delineations to the patients to ensure they match T2 resolution. As of now it is not yet completely clear how to represent the delineations within a patient or slice so the reading will be performed later (this is also not necessary for the EDA).

In [ ]:
patient_1_delineations = delineations["MARPROC007"]

# iterate over slices
delineated_slices = []
print(patient_1_delineations[0][0])
for i, slice in enumerate(np.rollaxis(patient_1_delineations[0][1][0], 2)):
    if 1 in slice:
        delineated_slices.append(i)

print("\nSlices that contain a delineation: ")
print(delineated_slices)



### Checking in how many cases the shape of the delineation of a patient matches the loaded axial t2 of that patient

In [ ]:
bad_delin_patients = set()

for patient_id, delineations_patient in delineations.items():
    delineations_shape_patient = delineations_patient[0][1][0].shape

    if patient_id in patients:
        t2_shape_patient = patients[patient_id].get_axialt2_image_array().shape

        if delineations_shape_patient != t2_shape_patient:
            bad_delin_patients.add(patient_id)
            print(patient_id, ": Shape of delineations and t2 doesnt match")
            print("dimensions of delineations: ", delineations_shape_patient)
            print("Axialt2 of patient:         ", t2_shape_patient)
            print()


In [ ]:
# MARPROC343 seems to have a weirdly transposed shape
# The amount of slices doesnt match however, so just leave it.
# tft_delin = delineations["MARPROC343"]
# tft = patients["MARPROC343"]
# print(tft_delin[0][1][0].shape)
# print(tft.get_axialt2_image_array().shape)

### Now to add the delineations to the patient objects
Each Patient will get a single 3d array representing the map of all the Gleason Pattern regions together. In this map for each voxel, 0 represents healthy tissue, 1 represents GG3, 2 represents GG4 and 3 represents Cribriform

In [ ]:
importlib.reload(sys.modules["patient"])
importlib.reload(sys.modules["read_scans_utils"])
import patient
from patient import Patient
import read_scans_utils

In [ ]:
read_scans_utils.combine_patients_delineations(patients, delineations)

In [ ]:
Patient.show_patient_delineation_slices(patients["MARPROC007"])
print()
# Patient.show_patient_delineation_slices(patients["MARPROC204"])

# print([delineation[0] for delineation in delineations["MARPROC204"]])


In [ ]:
# Lets check MARPROC204, the patient with 6 delineation files
delineated_slices_gg3 = []
delineated_slices_gg4 = []
delineated_slices_Cribriform = []

# for i, slice in enumerate(np.rollaxis(patients["MARPROC204"].region_delineation, 2)):
for i, slice in enumerate(np.rollaxis(patients["MARPROC007"].region_delineation, 2)):
    if 1 in slice:
        delineated_slices_gg3.append(i)
    if 2 in slice:
        delineated_slices_gg4.append(i)
    if 3 in slice:
        delineated_slices_Cribriform.append(i)

print(delineated_slices_gg3)
print(delineated_slices_gg4)
print(delineated_slices_Cribriform)


## Selecting slices from patients based on if they have a delineation available
Not all slices have delineations associated with them. Slices that don't, lack ground truth, since the lack of a delineation does not signify the absence of cancerous tissue. Thus only the slices that have ground truth are usable for model training. For every patient, the slices, and the numbers/indexes of the slices in segmentation that have a delineation will be collected. The indexes will then be used to collect associated slices from the other images of that patient. This will result in a number of quadruples per patient, consisting of a triple of slices for each of the input modalities (T2, ADC & perfusion) and a segmentation slice. These will form the input for the model.

The model expects a 3d image per patient per modality, so for each patient a 3d image of those aforementioned slices will be created (resulting in 4 images including the segmentation). This will be a subset of the complete image of that modality. nnUNet will train slice-wise using these, meaning it will take single slices across modalities (the quadruples).

In [ ]:
from nibabel.spatialimages import SpatialFirstSlicer
test = SpatialFirstSlicer(patients["MARPROC007"].adcmap)
# print(test)
# print(test.shape)

hoi = test[:, :, 5:10]
print(hoi)
print(hoi.shape)
print(hoi.get_fdata())


In [ ]:
plt.imshow(hoi.get_fdata()[:, :, 3])

In [ ]:
read_scans_utils.resize_dwis(patients)

In [ ]:
print(patients)

In [ ]:
patients["MARPROC007"].extract_slice_tuples()

### Exploring sitk, a commong library for working with multidimensional and multimodal medical imaging data

In [ ]:
plt.imshow(np.asanyarray(perffrac_p7.dataobj[:, :, 0]))
print(np.asanyarray(perffrac_p7.dataobj[:, :, 0]).shape)


In [ ]:
img_perf = sitk.GetImageFromArray(np.asanyarray(perffrac_p7.dataobj))
img_adc = sitk.GetImageFromArray(np.asanyarray(adcmap_p7.dataobj))
# img_perf.GetSize()
# img_perf.GetDepth()

In [ ]:
z = 0
slice = sitk.GetArrayViewFromImage(img_perf)[:, :, z]
plt.imshow(slice)

In [ ]:
def myshow(img):
    nda = sitk.GetArrayViewFromImage(img)
    plt.imshow(nda)

In [ ]:
# I have no idea what this does
myshow(img_perf[0, :, :] > 0)

### Possible image blending
simpleitk also contains functionality to blend segmentations with images (or contours of segmentations)

In [ ]:
def mask_image_multiply(mask, image):
    components_per_pixel = image.GetNumberOfComponentsPerPixel()
    if components_per_pixel == 1:
        return mask * image
    else:
        return sitk.Compose(
            [
                mask * sitk.VectorIndexSelectionCast(image, channel)
                for channel in range(components_per_pixel)
            ]
        )


def alpha_blend(image1, image2, alpha=0.5, mask1=None, mask2=None):
    """
    Alaph blend two images, pixels can be scalars or vectors.
    The alpha blending factor can be either a scalar or an image whose
    pixel type is sitkFloat32 and values are in [0,1].
    The region that is alpha blended is controled by the given masks.
    """

    if not mask1:
        mask1 = sitk.Image(image1.GetSize(), sitk.sitkFloat32) + 1.0
        mask1.CopyInformation(image1)
    else:
        mask1 = sitk.Cast(mask1, sitk.sitkFloat32)
    if not mask2:
        mask2 = sitk.Image(image2.GetSize(), sitk.sitkFloat32) + 1
        mask2.CopyInformation(image2)
    else:
        mask2 = sitk.Cast(mask2, sitk.sitkFloat32)
    # if we received a scalar, convert it to an image
    if type(alpha) != sitk.SimpleITK.Image:
        alpha = sitk.Image(image1.GetSize(), sitk.sitkFloat32) + alpha
        alpha.CopyInformation(image1)
    components_per_pixel = image1.GetNumberOfComponentsPerPixel()
    if components_per_pixel > 1:
        img1 = sitk.Cast(image1, sitk.sitkVectorFloat32)
        img2 = sitk.Cast(image2, sitk.sitkVectorFloat32)
    else:
        img1 = sitk.Cast(image1, sitk.sitkFloat32)
        img2 = sitk.Cast(image2, sitk.sitkFloat32)

    intersection_mask = mask1 * mask2

    intersection_image = mask_image_multiply(
        alpha * intersection_mask, img1
    ) + mask_image_multiply((1 - alpha) * intersection_mask, img2)
    return (
        intersection_image
        + mask_image_multiply(mask2 - intersection_mask, img2)
        + mask_image_multiply(mask1 - intersection_mask, img1)
    )

In [ ]:
blend = (alpha_blend(img_perf, img_adc), "alpha_blend_standard")

myshow(blend[0][0, :, :])


In [ ]:
myshow(img_perf[0, :, :])

In [ ]:
myshow(img_adc[0, :, :])
